In [1]:
import sys
! pip install pymongo pandas pyarrow


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from pymongo import MongoClient
import pandas as pd
import json

In [3]:
client = MongoClient('localhost', 27017)
db = client.data_managment

In [4]:
fnomceo_data= list(db.fnomceo_data.find())
fnomceo_df = pd.DataFrame(fnomceo_data)
print(fnomceo_df)

                             _id person_id    surname  \
0       65d13da6294d33d94a501435    399366    BADESSA   
1       65d13da6294d33d94a501436     33538    BADESSA   
2       65d13da6294d33d94a501437    325585    BADESSI   
3       65d13da6294d33d94a501438    343163    BADESSI   
4       65d13da6294d33d94a501439    554984    BADESSI   
...                          ...       ...        ...   
357079  65d13dac294d33d94a55870c    180274  BADARACCO   
357080  65d13dac294d33d94a55870d     73428  BADARACCO   
357081  65d13dac294d33d94a55870e    549817      BADAS   
357082  65d13dac294d33d94a55870f    483756      BADAS   
357083  65d13dac294d33d94a558710    218926      BADAS   

                     first_name        date_of_birth  \
0                       ANTONIO  1946-07-06 00:00:00   
1                     FRANCESCO  1967-01-01 00:00:00   
2                     FRANCESCA  1981-07-03 00:00:00   
3       FRANCESCO GIUSEPPE ALDO  1947-12-18 00:00:00   
4                       GIORGIO  19

In [5]:
fnofi_data= list(db.fnofi_data.find())
fnofi_df = pd.DataFrame(fnofi_data)
print(fnofi_df)

                            _id  person_id     surname  first_name  \
0      65d14075294d33d94a56ecb8     101222   Abbatelli      Chiara   
1      65d14075294d33d94a56ecb9      93494      Abbati     Tommaso   
2      65d14075294d33d94a56ecba      85313      Abbati   Francesca   
3      65d14075294d33d94a56ecbb     138494      Abbati      Fausta   
4      65d14075294d33d94a56ecbc     116220  Abbattista      Teresa   
...                         ...        ...         ...         ...   
48308  65d14075294d33d94a57a96c     200402   Arcangeli        Aldo   
48309  65d14075294d33d94a57a96d      86779      Arcara     Umberto   
48310  65d14075294d33d94a57a96e     204490      Arcari       Elisa   
48311  65d14075294d33d94a57a96f     173674      Arcari  Alessandro   
48312  65d14075294d33d94a57a970     128140      Arcaro    Gianluca   

            date_of_birth                 birth_place  \
0     1983-01-04 23:00:00                   Jesi (AN)   
1     1996-04-18 22:00:00         Bagno a Rip

In [6]:
alboweb_data= list(db.alboweb_data.find())
alboweb_df = pd.DataFrame(alboweb_data)
print(alboweb_df.head())

                        _id  person_id surname      first_name  \
0  65d14000294d33d94a558711     250493    Abbà         Roberta   
1  65d14000294d33d94a558712     131688    Abbà         Manuela   
2  65d14000294d33d94a558713     250498    Abbà         Giorgia   
3  65d14000294d33d94a558714     250491    Abbà  Elena Giovanna   
4  65d14000294d33d94a558715     154516   Abba'           Elena   

        date_of_birth      birth_place                    client_name  \
0 1977-06-05 22:00:00       Cuneo (CN)                          Cuneo   
1 1986-01-11 23:00:00       Cuneo (CN)                          Cuneo   
2 1983-10-16 23:00:00       Cuneo (CN)                          Cuneo   
3 1973-03-30 23:00:00  Savigliano (CN)                          Cuneo   
4 1966-02-13 23:00:00      Torino (TO)  Torino Aosta Alessandria Asti   

                                    special_register  
0  [{'board_name': 'Educazione Professionale', 'e...  
1  [{'board_name': 'Ortottica e Assistenza Oftalm...  

In [50]:
from datetime import datetime
import re

def prepare_for_integration(all_data):
    for data in all_data:
        data['iscrizioni'] = data.pop('special_register')
        for entry in data['iscrizioni']:
            entry['number'] = entry.pop('entry_number')
            entry['year'] = entry["inscripted_at"].year
        data['lauree'] = [{"name": entry['board_name']} for entry in data['iscrizioni']]
    return all_data

def replace_date_fnomceo(data):
    pattern = re.compile(r'\d{2}/\d{2}/\d{4}')
    for element in data:
        date_fields = ["date_of_birth", "last_update_date"]
        for field in date_fields:
            if field in element:
             element[field] = datetime.strptime(element[field], "%d/%m/%Y")
        if "iscrizioni" in element:
            for entry in element["iscrizioni"]:
                if pattern.match(entry["year"]):
                    entry["year"] = datetime.strptime(entry["year"], "%d/%m/%Y")
        if "lauree" in element:
            for entry in element["lauree"]:
                if pattern.match(entry["year"]):
                    entry["year"] = datetime.strptime(entry["year"], "%d/%m/%Y")
        # 
        if "abilitazioni" in element:
            for entry in element["abilitazioni"]:
                if int(entry["year"]) > 1000:
                    entry["year"] = datetime.strptime(entry["year"].strip(), "%Y")
        
        if "specializzazioni" in element:
            for entry in element["specializzazioni"]:
                if pattern.match(entry["year"]):
                    entry["year"] = datetime.strptime(entry["year"], "%d/%m/%Y")
        #     
    return data

def replace_date_fnofi(data):
    pattern = re.compile(r'\d{2}/\d{2}/\d{4}')
    for element in data:
        date_fields = ["date_of_birth"]
        for field in date_fields:
            if field in element:
             element[field] = datetime.strptime(element[field], "%Y-%m-%dT%H:%M:%S.%fZ")
        if "special_register" in element:
            for entry in element["special_register"]:
                if pattern.match(entry["inscripted_at"]):
                    entry["inscripted_at"] = datetime.strptime(entry["inscripted_at"], "%d/%m/%Y")
    return data

In [51]:
import json

# Read the first JSON file
with open("final_data/fnomceo/fnomceo_final_data.json", "r") as data:
    fnomceo_data = json.load(data)
prepared_fnomceo_data = replace_date_fnomceo(fnomceo_data)

# Read the second JSON file
with open("final_data/fnofi/fnofi_final_data.json", "r") as data:
    fnofi_data = json.load(data)
updated_data = replace_date_fnofi(fnofi_data)
prepared_fnofi_data = prepare_for_integration(updated_data)
    
with open("final_data/alboweb/alboweb_final_data_replaced_profession.json", "r") as data:
    alboweb_data = json.load(data)
updated_data = replace_date_fnofi(alboweb_data)
prepared_alboweb_data = prepare_for_integration(updated_data)

fnomceo_df = pd.DataFrame(prepared_fnomceo_data)
fnofi_df = pd.DataFrame(prepared_fnofi_data)
alboweb_df = pd.DataFrame(prepared_alboweb_data)

df_list = [fnomceo_df, fnofi_df, alboweb_df]

concatenated_df = pd.concat(df_list, keys=['fnomceo', 'fnofi', 'alboweb'], names=['source'])

concatenated_df.reset_index(inplace=True, level='source')


In [52]:
data_dict = concatenated_df.to_dict(orient='records')
with open('final_data/integrated_data/integrated.json', 'w') as json_file:
    json.dump(data_dict,json_file, indent=4, default=str)
#     
# with open('final_data/integrated_data/integrated.json', 'w', encoding='utf-8') as f:
#     json.dump(concatenated_df, f, ensure_ascii=False, indent=4,default=str)